In [1]:
import psycopg2
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Get database connection string from environment variable
DB_CONNECTION_STRING = os.getenv("DB_CONNECTION_STRING")
np.random.seed(1234)

# Создаем соединение с базой данных
engine = create_engine(DB_CONNECTION_STRING)

# Загружаем данные из таблиц user_data, post_text_df и feed_data
user_data = pd.read_sql("SELECT * FROM public.user_data", con=engine)
post_text_df = pd.read_sql("SELECT * FROM public.post_text_df", con=engine)
feed_data = pd.read_sql("SELECT * FROM public.feed_data ORDER BY timestamp DESC LIMIT 5000000", con=engine)

In [ ]:
import re
from transformers import RobertaModel
from transformers import AutoTokenizer
import torch
from tqdm import tqdm
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Move the model to the GPU

def preprocess_text(text):
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    # Remove leading and trailing whitespaces
    text = text.strip()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters except punctuation
    text = re.sub(r'[^a-zA-Z0-9.,!?\'"\s]', '', text)
    return text

def get_embeddings(text, model, tokenizer):
    
    encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_token_type_ids=False,
            return_tensors='pt'
        )
    encoding = {k: v.to(device) for k, v in encoding.items()}  # Move input tensors to the GPU

    with torch.no_grad():
        output = model(**encoding)
    return output.pooler_output.squeeze().detach().cpu().numpy()

embeddings = []
for text in tqdm(post_text_df['text']):
    if text.strip():  # Skip empty strings
        preprocessed_text = preprocess_text(text)
        embedding = get_embeddings(preprocessed_text, model, tokenizer)
        embeddings.append(embedding)

# Преобразование списка эмбеддингов в массив numpy
embeddings = np.array(embeddings)

# Создание отдельных столбцов для каждого эмбеддинга
embedding_columns = {}
for i in range(embeddings.shape[1]):
    column_name = f'embedding_{i}'
    embedding_columns[column_name] = embeddings[:, i]

# Создание нового датафрейма с отдельными столбцами для эмбеддингов
embedding_df = pd.DataFrame(embedding_columns)

# Объединение датафрейма эмбеддингов с другими признаками
merged_df = pd.concat([post_text_df, embedding_df], axis=1)

In [ ]:
# Объединяем данные в один DataFrame
user_feed = pd.merge(feed_data, user_data, on='user_id', how='left')
all_data = pd.merge(user_feed, merged_df, on='post_id', how='left')

def generate_new_target(target, action):
    if target == 1 or action == 'like':
        return 1
    else:
        return 0

all_data['target'] = all_data.apply(lambda row: generate_new_target(row['target'], row['action']), axis=1)

# Создаем признаки
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])
all_data['day_of_week'] = all_data['timestamp'].dt.dayofweek
all_data['hour'] = all_data['timestamp'].dt.hour
all_data['time_slot'] = all_data['day_of_week'].astype(str) + '_' + all_data['hour'].astype(str)

train = all_data.drop(['action', 'timestamp'], axis=1)

In [ ]:
# Generate User Features
# Add user mean to train
user_means = train.groupby('user_id')['target'].mean()
train['user_means'] = train['user_id'].map(user_means)
# Target by'topic'
unique_topics = train['topic'].unique()
target_column_names = []
for topic in unique_topics:
    topic_col_name = f"{topic}_target"
    target_column_names.append(topic_col_name)
    user_target_by_topic = train[train['target'] == 1].groupby(['user_id', 'topic']).size().reset_index(name='temp')
    user_target_by_topic = user_target_by_topic[user_target_by_topic['topic'] == topic].rename(columns={'temp': topic_col_name})
    user_target_by_topic = user_target_by_topic[['user_id', topic_col_name]]
    train = pd.merge(train, user_target_by_topic, on='user_id', how='left')
    train[topic_col_name].fillna(0, inplace=True)
    
# Generate user table and preload to db
user_columns = ['user_id', 'user_means'] + target_column_names

user_features = train[user_columns].drop_duplicates(subset=['user_id'])
user_data_enriched = pd.merge(user_data, user_features, on='user_id', how='left')

for col in user_columns:
       user_data_enriched[col] = user_data_enriched[col].fillna(user_data_enriched[col].mean())

205

In [ ]:
# Calculate total likes for each post
total_likes = train.groupby('post_id')['target'].sum().reset_index()
total_likes.columns = ['post_id', 'total_likes']
train = train.merge(total_likes, on='post_id', how='left')
# Calculate post CTR (Click-Through Rate)
post_ctr = train.groupby('post_id')['target'].mean().reset_index()
post_ctr.columns = ['post_id', 'post_ctr']
train = train.merge(post_ctr, on='post_id', how='left')

# Prepare post features DataFrame for loading to DB
post_features_columns = ['post_id'] + ['total_likes', 'post_ctr']
post_features = train[post_features_columns].drop_duplicates(subset=['post_id'])
post_data_enriched = pd.merge(post_text_df, post_features, on='post_id', how='left')

for col in post_features_columns:
       post_data_enriched[col] = post_data_enriched[col].fillna(post_data_enriched[col].mean())


In [ ]:
post_data_enriched.to_sql('martynov_post_features_lesson_22_posts', con=engine, if_exists='replace', index=False)

23